# BentoML Example: H2O Classification


[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. 

This notebook demonstrates how to use BentoML to __turn a H2O model into a docker image containing a REST API server__ serving this model, as well as distributing your model as a command line tool or a pip-installable PyPI package.

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=h2o&ea=h2o-prostate-cancer-classification&dt=h2o-prostate-cancer-classification)

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install bentoml
!pip install h2o

In [6]:
import h2o
import bentoml

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "9.0.1"; Java(TM) SE Runtime Environment (build 9.0.1+11); Java HotSpot(TM) 64-Bit Server VM (build 9.0.1+11, mixed mode)
  Starting server from /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpkq2pnyhu
  JVM stdout: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpkq2pnyhu/h2o_bozhaoyu_started_from_python.out
  JVM stderr: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpkq2pnyhu/h2o_bozhaoyu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,9 months and 26 days !!!
H2O cluster name:,H2O_from_python_bozhaoyu_7xlm4g
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


This show case considers prostate cancer data and tries to find an algorithm to prognose a certain phase of cancer. The dataset was collected at the Ohio State University Comprehensive Cancer Center and includes demographic and medical data from each of the 380 patients as well as a classifier identifying if the patients tumor has already penetrated the prostatic capsule. This latter event is a clear sign for an advanced cancer state and also helps the doctor to decide on biopsy and treatment methods.

In this show case a deep learning algorithm is used to classify the tumors of the patients into 'penetrating prostatic capsule' and 'not penetrating prostatic capsule'. 

# Prepare Dataset & Model Training

In [7]:
prostate = h2o.import_file(path="https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
prostate.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Rows:380
Cols:9




,ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON
type,int,int,int,int,int,int,real,real,int
mins,1.0,0.0,43.0,0.0,1.0,1.0,0.3,0.0,0.0
mean,190.5,0.4026315789473684,66.03947368421049,1.0868421052631572,2.2710526315789488,1.1078947368421048,15.408631578947375,15.812921052631573,6.3842105263157904
maxs,380.0,1.0,79.0,2.0,4.0,2.0,139.7,97.6,9.0
sigma,109.84079387914127,0.4910743389630552,6.527071269173311,0.3087732580252793,1.0001076181502861,0.3106564493514939,19.99757266856046,18.347619967271175,1.0919533744261092
zeros,0,227,0,3,0,0,0,167,2
missing,0,0,0,0,0,0,0,0,0
0,1.0,0.0,65.0,1.0,2.0,1.0,1.4,0.0,6.0
1,2.0,0.0,72.0,1.0,3.0,2.0,6.7,0.0,7.0
2,3.0,0.0,70.0,1.0,1.0,2.0,4.9,0.0,6.0


In [8]:
# import the deep learning estimator module
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
# transform the target variable into a factor
prostate["CAPSULE"] = prostate["CAPSULE"].asfactor()
# construct and define the estimator object 
model = H2ODeepLearningEstimator(activation = "Tanh", hidden = [10, 10, 10], epochs = 100)
# train the model on the whole prostate dataset
model.train(x = list(set(prostate.columns) - set(["ID","CAPSULE"])), y ="CAPSULE", training_frame = prostate)
model.show()

deeplearning Model Build progress: |██████████████████████████████████████| 100%
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_model_python_1581550864917_1


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.13742235604104663
RMSE: 0.3707052144778201
LogLoss: 0.418901646759126
Mean Per-Class Error: 0.18793009127292626
AUC: 0.8853473841812789
pr_auc: 0.8245437620191027
Gini: 0.7706947683625578
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.30356438890837084: 


,0,1,Error,Rate
0,158.0,69.0,0.304,(69.0/227.0)
1,11.0,142.0,0.0719,(11.0/153.0)
Total,169.0,211.0,0.2105,(80.0/380.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3035644,0.7802198,210.0
max f2,0.2321613,0.8698225,232.0
max f0point5,0.6135485,0.7703281,136.0
max accuracy,0.6135485,0.8052632,136.0
max precision,0.9957857,1.0,0.0
max recall,0.0980649,1.0,292.0
max specificity,0.9957857,1.0,0.0
max absolute_mcc,0.3035644,0.6159649,210.0
max min_per_class_accuracy,0.4725253,0.7973856,166.0
max mean_per_class_accuracy,0.3035644,0.8120699,210.0


Gains/Lift Table: Avg response rate: 40.26 %, avg score: 43.75 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0105263,0.9850611,2.4836601,2.4836601,1.0,0.9897941,1.0,0.9897941,0.0261438,0.0261438,148.3660131,148.3660131
,2,0.0210526,0.9794845,2.4836601,2.4836601,1.0,0.9830457,1.0,0.9864199,0.0261438,0.0522876,148.3660131,148.3660131
,3,0.0315789,0.9636331,2.4836601,2.4836601,1.0,0.9703565,1.0,0.9810655,0.0261438,0.0784314,148.3660131,148.3660131
,4,0.0421053,0.9486374,2.4836601,2.4836601,1.0,0.9545958,1.0,0.9744480,0.0261438,0.1045752,148.3660131,148.3660131
,5,0.05,0.9408173,1.6557734,2.3529412,0.6666667,0.9469043,0.9473684,0.9700990,0.0130719,0.1176471,65.5773420,135.2941176
,6,0.1,0.9031731,2.3529412,2.3529412,0.9473684,0.9266963,0.9473684,0.9483977,0.1176471,0.2352941,135.2941176,135.2941176
,7,0.15,0.8652018,2.0915033,2.2657952,0.8421053,0.8848821,0.9122807,0.9272258,0.1045752,0.3398693,109.1503268,126.5795207
,8,0.2,0.8293360,1.8300654,2.1568627,0.7368421,0.8468148,0.8684211,0.9071231,0.0915033,0.4313725,83.0065359,115.6862745
,9,0.3,0.7140976,1.6993464,2.0043573,0.6842105,0.7817978,0.8070175,0.8653480,0.1699346,0.6013072,69.9346405,100.4357298
,10,0.4,0.5480790,1.3071895,1.8300654,0.5263158,0.6254777,0.7368421,0.8053804,0.1307190,0.7320261,30.7189542,83.0065359



Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
,2020-02-12 15:41:10,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan
,2020-02-12 15:41:11,1.369 sec,18627 obs/sec,10.0,1,3800.0,0.4276705,0.5659435,0.2395536,0.8186346,0.7723048,2.4836601,0.2473684
,2020-02-12 15:41:11,1.557 sec,101604 obs/sec,100.0,10,38000.0,0.3707052,0.4189016,0.4286433,0.8853474,0.8245438,2.4836601,0.2105263


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PSA,1.0,1.0,0.1684290
AGE,0.9730824,0.9730824,0.1638953
VOL,0.9568093,0.9568093,0.1611544
GLEASON,0.8978918,0.8978918,0.1512310
DPROS,0.8264121,0.8264121,0.1391918
DCAPS,0.7811980,0.7811980,0.1315764
RACE,0.5018259,0.5018259,0.0845220


In [9]:
predictions=model.predict(prostate)
predictions.show()

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
0,0.759182,0.240818
1,0.20655,0.79345
0,0.92885,0.0711497
1,0.529636,0.470364
0,0.994685,0.00531483
1,0.0712494,0.928751
0,0.719996,0.280004
0,0.887014,0.112986
0,0.741257,0.258743
0,0.946604,0.0533964


## Define BentoService for model serving

In [23]:
%%writefile h2o_model_service.py
import pandas as pd
import h2o
import bentoml
from bentoml.artifact import H2oModelArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([H2oModelArtifact('model')])
@bentoml.env(
    pip_dependencies=['pandas', 'h2o==3.24.0.2'],
    conda_channels=['h2oai'],
    conda_dependencies=['h2o==3.24.0.2']
)
class H2oModelService(bentoml.BentoService):

    @bentoml.api(DataframeHandler)
    def predict(self, df):     
        hf = h2o.H2OFrame(df)
        predictions = self.artifacts.model.predict(hf)
        return predictions.as_data_frame()

Overwriting h2o_model_service.py


## Save BentoService to file archive

In [24]:
# 1) import the custom BentoService defined above
from h2o_model_service import H2oModelService

# 2) `pack` it with required artifacts
bento_svc = H2oModelService()
bento_svc.pack('model', model)

# 3) save your BentoSerivce
saved_path = bento_svc.save()
print(saved_path)

running sdist
running egg_info
writing BentoML.egg-info/PKG-INFO
writing dependency_links to BentoML.egg-info/dependency_links.txt
writing entry points to BentoML.egg-info/entry_points.txt
writing requirements to BentoML.egg-info/requires.txt
writing top-level names to BentoML.egg-info/top_level.txt
reading manifest file 'BentoML.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'


no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'
no previously-included directories found matching 'scripts'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.2+7.g1d9fac1
creating BentoML-0.6.2+7.g1d9fac1/BentoML.egg-info
creating BentoML-0.6.2+7.g1d9fac1/bentoml
creating BentoML-0.6.2+7.g1d9fac1/bentoml/artifact
creating BentoML-0.6.2+7.g1d9fac1/bentoml/bundler
creating BentoML-0.6.2+7.g1d9fac1/bentoml/cli
creating BentoML-0.6.2+7.g1d9fac1/bentoml/clipper
creating BentoML-0.6.2+7.g1d9fac1/bentoml/configuration
creating BentoML-0.6.2+7.g1d9fac1/bentoml/deployment
creating BentoML-0.6.2+7.g1d9fac1/bentoml/deployment/aws_lambda
creating BentoML-0.6.2+7.g1d9fac1/bentoml/deployment/sagemaker
creating BentoML-0.6.2+7.g1d9fac1/bentoml/handlers
creating BentoML-0.6.2+7.g1d9fac1/bentoml/marshal
creating BentoML-0.6.2+7.g1d9fac1/bentoml/migrations
creating BentoML-0.6.2+7.g1d9fac1/bentoml/migrations/versions
creating BentoML-0.6.2+7.g1d9fac1/bentoml/proto
creating BentoML-0.6.2+7.g1d9fac1/bentoml/repository
creating BentoML-0.6.2+7.g1d9fac1/bentoml/server
creating BentoML-0.6.2+7.g1d9fac1/bentoml/server/static
creating BentoML-0

copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/marshal_server.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/middlewares.py -> BentoML-0.6.2+7.g1d9fac1/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.6.2+7.g1d9f

## Load BentoService from archive

In [26]:
import bentoml
import pandas as pd

# Load saved BentoService archive from file directory
loaded_bento_svc = bentoml.load(saved_path)

# Access the predict function of loaded BentoService
df = pd.read_csv("https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv")
loaded_bento_svc.predict(df)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,9 mins 57 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,9 months and 26 days !!!
H2O cluster name:,H2O_from_python_bozhaoyu_7xlm4g
H2O cluster total nodes:,1
H2O cluster free memory:,4.000 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


,predict,p0,p1
0,0,0.759182,0.240818
1,1,0.206550,0.793450
2,0,0.928850,0.071150
3,1,0.529636,0.470364
4,0,0.994685,0.005315
5,1,0.071249,0.928751
6,0,0.719996,0.280004
7,0,0.887014,0.112986
8,0,0.741257,0.258743
9,0,0.946604,0.053396


## Use BentoML CLI with BentoServices

**Use `bentoml get <service name>` to list all of the service's versions**

In [27]:
!bentoml get H2oModelService

BENTO_SERVICE                          AGE                          APIS                       ARTIFACTS
H2oModelService:20200212154840_9B4F59  2 minutes and 8.83 seconds   predict<DataframeHandler>  model<H2oModelArtifact>
H2oModelService:20200212154125_149345  9 minutes and 23.91 seconds  predict<DataframeHandler>  model<H2oModelArtifact>
H2oModelService:20200127130336_743DA0  2 weeks and 2 days           predict<DataframeHandler>  model<H2oModelArtifact>


`bentoml get <service name>:<version>` will display metadata and additional information of the specific version

In [28]:
!bentoml get H2oModelService:20200212154840_9B4F59

{
  "name": "H2oModelService",
  "version": "20200212154840_9B4F59",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/H2oModelService/20200212154840_9B4F59"
  },
  "bentoServiceMetadata": {
    "name": "H2oModelService",
    "version": "20200212154840_9B4F59",
    "createdAt": "2020-02-12T23:49:05.422467Z",
    "env": {
      "condaEnv": "name: bentoml-H2oModelService\nchannels:\n- defaults\n- h2oai\ndependencies:\n- python=3.7.3\n- pip\n- h2o\n",
      "pipDependencies": "bentoml==0.6.2\nh2o\npandas",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "H2oModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "output_orient": "records",
          "orient": "records",
          "typ": "frame",
          "input_dtypes": null
        }
      }
    ]
  }
}


Use `bentoml run` command to get prediction result from the saved BentoService

In [29]:
!bentoml run H2oModelService:20200212154840_9B4F59 predict \
    --input https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv

[2020-02-12 15:51:38,487] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-12 15:51:38,502] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+7.g1d9fac1
[2020-02-12 15:51:38,578] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

**With `bentoml serve` You can quickly start a realtime serving API for testing and demo**

In [16]:
!bentoml serve {saved_path}

[2020-02-12 15:42:39,257] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-12 15:42:39,271] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+7.g1d9fac1
[2020-02-12 15:42:39,310] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

#### Send prediction request to REST API server

Run the following command in terminal to make a HTTP request to the API server:
```bash
curl -i \
--header "Content-Type: text/csv" \
--request POST \
--data 'ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON\n
1,0,65,1,2,1,1.4,0,6\n
2,0,72,1,3,2,6.7,0,7\n' \
localhost:5000/predict
```


# "pip install" a BentoService archive

BentoML user can directly pip install saved BentoService with `pip install $SAVED_PATH`,  and use it as a regular python package.

In [17]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/H2oModelService/20200212154125_149345


  Created wheel for H2oModelService: filename=H2oModelService-20200212154125.post149345-py3-none-any.whl size=145099 sha256=1d6b9e04e439f68f8cb9415113ab7dafa0a326dc89d16705bba00f15a6111589
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-a2nwrld4/wheels/86/26/b4/8a55f14f1443047214b64792c8eb5088a523b8ae365c272b00
Successfully built H2oModelService
  Attempting uninstall: H2oModelService
    Found existing installation: H2oModelService 20200127130336-743DA0
    Uninstalling H2oModelService-20200127130336-743DA0:
      Successfully uninstalled H2oModelService-20200127130336-743DA0


In [18]:
# Your bentoML model class name will become packaged name
import H2oModelService

ms = H2oModelService.load() # call load to ensure all artifacts are loaded
ms.predict(pd.read_csv('https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv'))

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 min 52 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,9 months and 26 days !!!
H2O cluster name:,H2O_from_python_bozhaoyu_7xlm4g
H2O cluster total nodes:,1
H2O cluster free memory:,4.000 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


,predict,p0,p1
0,0,0.759182,0.240818
1,1,0.206550,0.793450
2,0,0.928850,0.071150
3,1,0.529636,0.470364
4,0,0.994685,0.005315
5,1,0.071249,0.928751
6,0,0.719996,0.280004
7,0,0.887014,0.112986
8,0,0.741257,0.258743
9,0,0.946604,0.053396


## Use installed BentoService as CLI tool

After install BentoService as PyPI package, you also have access to generated BentoService CLI

In [19]:
!H2oModelService --help

Usage: H2oModelService [OPTIONS] COMMAND [ARGS]...

  BentoML CLI tool

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  info            List APIs
  open-api-spec   Display OpenAPI/Swagger JSON specs
  run             Run API function
  serve           Start local rest server
  serve-gunicorn  Start local gunicorn server


In [20]:
!H2oModelService info

[2020-02-12 15:43:05,821] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-12 15:43:05,832] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+7.g1d9fac1
{
  "name": "H2oModelService",
  "version": "20200212154125_149345",
  "created_at": "2020-02-12T23:41:50.347171Z",
  "env": {
    "conda_env": "name: bentoml-H2oModelService\nchannels:\n- defaults\n- h2oai\ndependencies:\n- pyt

In [16]:
!H2oModelService run predict --input https://raw.githubusercontent.com/multicode/h2o-notebook/master/prostate.csv

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
--------------------------  ---------------------------------------------------
H2O cluster uptime:         10 mins 55 secs
H2O cluster timezone:       America/Los_Angeles
H2O data parsing timezone:  UTC
H2O cluster version:        3.24.0.2
H2O cluster version age:    9 months and 10 days !!!
H2O cluster name:           H2O_from_python_bozhaoyu_t0tjgs
H2O cluster total nodes:    1
H2O cluster free memory:    3.929 Gb
H2O cluster total cores:    8
H2O cluster allowed cores:  8
H2O cluster status:         locked, healthy
H2O connection url:         http://localhost:54321
H2O connection proxy:
H2O internal security:      False
H2O API Extensions:         Amazon S3, XGBoost, Algos, AutoML, Core V3, Core V4
Python version:             3.7.3 final
--------------------------  ---------------------------------------------------
Parse progress: |█████████████████████████████████████████████████████████| 100


## Containerize REST API server with Docker

** _Note: `docker` is not available when running in Google Colaboratory_

#### Build the docker image

In [31]:
!cd {saved_path} && docker build -t h2o-model .

Sending build context to Docker daemon  1.121MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Using cache
 ---> 28172be83c07
Step 3/12 : EXPOSE 5000
 ---> Using cache
 ---> 840844d191d4
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 243c05e712f3
Step 5/12 : RUN conda install pip numpy scipy       && pip install gunicorn
 ---> Using cache
 ---> 8fab95ab34fc
Step 6/12 : COPY . /bento
 ---> 9e91f805b215
Step 7/12 : WORKDIR /bento
 ---> Running in 1c99f57aa9c2
Removing intermediate container 1c99f57aa9c2
 ---> aca013715929
Step 8/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in a1ca39c32171
Removing intermediate container a1ca39c32171
 ---> 2ea3b0b46eb5
Step 9/12 : RUN conda env update -n base -f /bento/environment.yml
 ---

  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp37-cp37m-linux_x86_64.whl size=1223691 sha256=9f1d0b7849433aa6e27c78afdde6a134ba84bfbb36dedb52f897048b83c57a1f
  Stored in directory: /root/.cache/pip/wheels/b9/ba/77/163f10f14bd489351530603e750c195b0ceceed2f3be2b32f1
  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=92cc432fcfd9befe9897ccea4aaacf610aa3e891fc115be6417694d11f40f9aa
  Stored in directory: /root/.cache/pip/wheels/fa/7f/fd/92ccdbb9d1a65486406e0363d2ba5b4ce52f400a915f602ecb
  Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157563 sha256=2705c8362c60608627fc5421905660a533e863bab2023269d5149890290fb42e
  Stored in directory: /root/.cache/pip/wheels/33/a9/f9/a53f885636269db5b76cf7afa3a1ab86d9d2fe96610d09274e
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491056 sha256=c362ca824f59f8ae9fe324924806de9ae1eea2bb03ac7c7d8797c178dea72801
  Stored in director

  Created wheel for BentoML: filename=BentoML-0.6.2+7.g1d9fac1-py3-none-any.whl size=513799 sha256=c7544e5d3fc0579ae26af94a6529097a9bc80b3eec8d3053d17323a36668a0cd
  Stored in directory: /root/.cache/pip/wheels/0a/0d/95/dfb521b2558409acf97e07ad18c855ea151e092f32ce3bd013
Successfully built BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.6.2
    Uninstalling BentoML-0.6.2:
      Successfully uninstalled BentoML-0.6.2
Removing intermediate container 091d53e394ea
 ---> 05f9a202c3b6
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in 83a2e7a9387e
Removing intermediate container 83a2e7a9387e
 ---> c95da2674f51
Successfully built c95da2674f51
Successfully tagged h2o-model:latest


#### Run the server with docker image

In [ ]:
!docker run -p 5000:5000 h2o-model

# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this demo, we are going to deploy to AWS Sagemaker

In [33]:
!bentoml sagemaker deploy first-h2o -b H2oModelService:20200212154840_9B4F59 --api-name predict

Deploying Sagemaker deployment -[2020-02-12 16:09:49,233] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-12 16:09:49,234] INFO - 

[2020-02-12 16:09:49,234] INFO -  ---> 406f2b43ea59

[2020-02-12 16:09:49,234] INFO - Step 2/11 : EXPOSE 8080
[2020-02-12 16:09:49,234] INFO - 

|[2020-02-12 16:09:49,412] INFO -  ---> Running in 80c56f62974c

-[2020-02-12 16:09:49,635] INFO -  ---> 85b4bb5fff81

[2020-02-12 16:09:49,635] INFO - Step 3/11 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential     && apt-get install -y nginx      && rm -rf /var/lib/apt/lists/*
[2020-02-12 16:09:49,635] INFO - 

|[2020-02-12 16:09:49,772] INFO -  ---> Running in 8caf67652239

\[2020-02-12 16:09:50,323] INFO - + apt-get update

/[2020-02-12 16:09:50,499] INFO - Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]

[2020-02-12 16:09:50,536] INFO - Get:2 http://deb.debian.org/debian

/[2020-02-12 16:10:02,377] INFO - (Reading database ... 90%
[2020-02-12 16:10:02,393] INFO - (Reading database ... 95%
(Reading database ... 12557 files and directories currently installed.)

[2020-02-12 16:10:02,410] INFO - Preparing to unpack .../00-binutils-common_2.31.1-16_amd64.deb ...

[2020-02-12 16:10:02,412] INFO - Unpacking binutils-common:amd64 (2.31.1-16) ...

-[2020-02-12 16:10:02,686] INFO - Selecting previously unselected package libbinutils:amd64.

[2020-02-12 16:10:02,687] INFO - Preparing to unpack .../01-libbinutils_2.31.1-16_amd64.deb ...

[2020-02-12 16:10:02,694] INFO - Unpacking libbinutils:amd64 (2.31.1-16) ...

/[2020-02-12 16:10:02,806] INFO - Selecting previously unselected package binutils-x86-64-linux-gnu.

[2020-02-12 16:10:02,808] INFO - Preparing to unpack .../02-binutils-x86-64-linux-gnu_2.31.1-16_amd64.deb ...

[2020-02-12 16:10:02,811] INFO - Unpacking binutils-x86-64-linux-gnu (2.31.1-16) ...

-[2020-02-12 16:10:03,105] INFO - Selecting previousl

|[2020-02-12 16:10:08,642] INFO - Selecting previously unselected package libpq-dev.

[2020-02-12 16:10:08,644] INFO - Preparing to unpack .../33-libpq-dev_11.6-0+deb10u1_amd64.deb ...

[2020-02-12 16:10:08,647] INFO - Unpacking libpq-dev (11.6-0+deb10u1) ...

[2020-02-12 16:10:08,699] INFO - Setting up binutils-common:amd64 (2.31.1-16) ...

[2020-02-12 16:10:08,708] INFO - Setting up libpq5:amd64 (11.6-0+deb10u1) ...

[2020-02-12 16:10:08,715] INFO - Setting up linux-libc-dev:amd64 (4.19.98-1) ...

\[2020-02-12 16:10:08,722] INFO - Setting up libpq-dev (11.6-0+deb10u1) ...

[2020-02-12 16:10:08,730] INFO - Setting up libgomp1:amd64 (8.3.0-6) ...

[2020-02-12 16:10:08,737] INFO - Setting up libasan5:amd64 (8.3.0-6) ...

[2020-02-12 16:10:08,745] INFO - Setting up make (4.2.1-1.2) ...

[2020-02-12 16:10:08,752] INFO - Setting up libmpfr6:amd64 (4.0.2-1) ...

[2020-02-12 16:10:08,761] INFO - Setting up libquadmath0:amd64 (8.3.0-6) ...

[2020-02-12 16:10:08,769] INFO - Setting up libmpc

[2020-02-12 16:10:12,681] INFO - Get:26 http://deb.debian.org/debian buster/main amd64 nginx-full amd64 1.14.2-2+deb10u1 [501 kB]

\[2020-02-12 16:10:12,833] INFO - Get:27 http://deb.debian.org/debian buster/main amd64 nginx all 1.14.2-2+deb10u1 [88.3 kB]

/[2020-02-12 16:10:13,025] INFO - debconf: delaying package configuration, since apt-utils is not installed

[2020-02-12 16:10:13,060] INFO - Fetched 7787 kB in 2s (3207 kB/s)

[2020-02-12 16:10:13,089] INFO - Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 
(Reading database ... 75% INFO - (Reading database ... 5%
[2020-02-12 16:10:13,094] INFO - (Reading database ... 80%
[2020-02-12 16:10:13,096] INFO - (Reading database ... 85%
[2020-02-12 16:10:13,097] INFO - (Reading database ... 90%
[2020-02-12 16:10:13,099] INFO - (Reading database ... 95%
(Reading database ... 16007 files and directories currently installed.)

[2020-02-12 16:10:13,102] INFO - Preparing to unpack .../00-fonts-dejavu-core_2.37

-[2020-02-12 16:10:15,030] INFO - Selecting previously unselected package nginx.

[2020-02-12 16:10:15,032] INFO - Preparing to unpack .../26-nginx_1.14.2-2+deb10u1_all.deb ...

[2020-02-12 16:10:15,034] INFO - Unpacking nginx (1.14.2-2+deb10u1) ...

[2020-02-12 16:10:15,076] INFO - Setting up libxpm4:amd64 (1:3.5.12-1) ...

/[2020-02-12 16:10:15,085] INFO - Setting up nginx-common (1.14.2-2+deb10u1) ...

|[2020-02-12 16:10:15,233] INFO - debconf: unable to initialize frontend: Dialog
debconf: (TERM is not set, so the dialog frontend is not usable.)
debconf: falling back to frontend: Readline

-[2020-02-12 16:10:15,485] INFO - Setting up libjbig0:amd64 (2.1-3.1+b2) ...

/[2020-02-12 16:10:15,495] INFO - Setting up libjpeg62-turbo:amd64 (1:1.5.2-2+b1) ...

[2020-02-12 16:10:15,502] INFO - Setting up libnginx-mod-http-auth-pam (1.14.2-2+deb10u1) ...

[2020-02-12 16:10:15,534] INFO - Setting up libpng16-16:amd64 (1.6.36-6) ...

[2020-02-12 16:10:15,545] INFO - Setting up libwebp6:amd

...

scipy-1.4.1          | 14.5 MB   | ####9      |  50% 
scipy-1.4.1          | 14.5 MB   | #####6     |  56% 
scipy-1.4.1          | 14.5 MB   | ######2    |  62% 
scipy-1.4.1          | 14.5 MB   | ######9    |  69% 
scipy-1.4.1          | 14.5 MB   | #######5   |  76% 
scipy-1.4.1          | 14.5 MB   | ########2  |  82% 
scipy-1.4.1          | 14.5 MB   | ########7  |  88% 
scipy-1.4.1          | 14.5 MB   | #########5 |  96% 
scipy-1.4.1          | 14.5 MB   | ########## | 100% 
[2020-02-12 16:10:47,798] INFO - 
ca-certificates-2020 | 125 KB    |            |   0% 
ca-certificates-2020 | 125 KB    | ########## | 100% 
[2020-02-12 16:10:47,830] INFO - 
mkl-service-2.3.0    | 218 KB    |            |   0% 
mkl-service-2.3.0    | 218 KB    | ########## | 100% 
[2020-02-12 16:10:47,888] INFO - 
openssl-1.1.1d       | 2.5 MB    |            |   0% 
openssl-1.1.1d       | 2.5 MB    | ###1       |  32% 
openssl-1.1.1d       | 2.5 MB    | ######3    |  63% 
openssl-1.1.1d       | 2.5 MB    |

h2o-3.18.0.2         | 118.2 MB  | #####8     |  58% 
h2o-3.18.0.2         | 118.2 MB  | #####8     |  59% 
h2o-3.18.0.2         | 118.2 MB  | #####9     |  59% 
h2o-3.18.0.2         | 118.2 MB  | ######     |  60% 
h2o-3.18.0.2         | 118.2 MB  | ######     |  61% 
h2o-3.18.0.2         | 118.2 MB  | ######1    |  62% 
h2o-3.18.0.2         | 118.2 MB  | ######2    |  62% 
h2o-3.18.0.2         | 118.2 MB  | ######3    |  63% 
h2o-3.18.0.2         | 118.2 MB  | ######3    |  64% 
h2o-3.18.0.2         | 118.2 MB  | ######4    |  64% 
h2o-3.18.0.2         | 118.2 MB  | ######4    |  65% 
h2o-3.18.0.2         | 118.2 MB  | ######5    |  66% 
h2o-3.18.0.2         | 118.2 MB  | ######6    |  66% 
h2o-3.18.0.2         | 118.2 MB  | ######6    |  67% 
h2o-3.18.0.2         | 118.2 MB  | ######7    |  68% 
h2o-3.18.0.2         | 118.2 MB  | ######8    |  68% 
h2o-3.18.0.2         | 118.2 MB  | ######9    |  70% 
h2o-3.18.0.2         | 118.2 MB  | #######    |  70% 
h2o-3.18.0.2         | 118.2

/[2020-02-12 16:12:14,827] INFO -  ---> 8bbaef7f9ac1

[2020-02-12 16:12:14,829] INFO - Step 8/11 : RUN pip install -r /opt/program/requirements.txt
[2020-02-12 16:12:14,829] INFO - 

|[2020-02-12 16:12:14,973] INFO -  ---> Running in f87fe33012be

-[2020-02-12 16:12:16,383] INFO - Collecting bentoml==0.6.2

/[2020-02-12 16:12:16,448] INFO -   Downloading BentoML-0.6.2-py3-none-any.whl (554 kB)

|[2020-02-12 16:12:16,981] INFO - Collecting pandas

[2020-02-12 16:12:16,991] INFO -   Downloading pandas-1.0.1-cp37-cp37m-manylinux1_x86_64.whl (10.1 MB)

\[2020-02-12 16:12:18,757] INFO - Collecting h2o==3.24.0.2

[2020-02-12 16:12:18,767] INFO -   Downloading h2o-3.24.0.2.tar.gz (121.0 MB)

|[2020-02-12 16:12:41,085] INFO - Collecting flask

[2020-02-12 16:12:41,095] INFO -   Downloading Flask-1.1.1-py2.py3-none-any.whl (94 kB)

\[2020-02-12 16:12:41,206] INFO - Collecting alembic

[2020-02-12 16:12:41,221] INFO -   Downloading alembic-1.4.0.tar.gz (1.1 MB)

\[2020-02-12 16:12:41,96

|[2020-02-12 16:13:02,375] INFO -   Building wheel for alembic (setup.py): finished with status 'done'

[2020-02-12 16:13:02,377] INFO -   Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157563 sha256=7a0f41f5c4b30c002506a961c1048a8aa99816bd39b6110c1195f6fce9486b38
  Stored in directory: /root/.cache/pip/wheels/33/a9/f9/a53f885636269db5b76cf7afa3a1ab86d9d2fe96610d09274e

[2020-02-12 16:13:02,379] INFO -   Building wheel for tabulate (setup.py): started

-[2020-02-12 16:13:02,649] INFO -   Building wheel for tabulate (setup.py): finished with status 'done'

[2020-02-12 16:13:02,650] INFO -   Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23273 sha256=5f4e865a5a3af1eb4ca8eea0d0095ae6601478e75f5a5caddee7fd0f9cd7000a
  Stored in directory: /root/.cache/pip/wheels/09/b6/7e/08b4ee715a1239453e89a59081f0ac369a9036f232e013ecd8

[2020-02-12 16:13:02,652] INFO -   Building wheel for cerberus (setup.py): started

\[2020-02-12 16:13:02,943] 

}


In [34]:
!bentoml sagemaker get first-h2o

{
  "namespace": "bobo",
  "name": "first-h2o",
  "spec": {
    "bentoName": "H2oModelService",
    "bentoVersion": "20200212154840_9B4F59",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-first-h2o",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-first-h2o",
      "EndpointConfigName": "bobo-first-h2o-H2oModelService-20200212154840-9B4F59",
      "ProductionVariants": [
        {
          "VariantName": "bobo-first-h2o-H2oModelService-20200212154840-9B4F59",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/h2omodelservice-sagemaker:20200212154840_9B4F59",
              "ResolvedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/h2omodelservice-sagemake

In [35]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name bobo-first-h2o \
--body 'ID,CAPSULE,AGE,RACE,DPROS,DCAPS,PSA,VOL,GLEASON\n1,0,65,1,2,1,1.4,0,6\n2,0,72,1,3,2,6.7,0,7\n' \
--content-type 'text/csv' output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-first-h2o-H2oModelService-20200212154840-9B4F59"
}
[]

In [36]:
!bentoml sagemaker list

NAME       NAMESPACE    PLATFORM       BENTO_SERVICE                          STATUS    AGE
first-h2o  bobo         aws-sagemaker  H2oModelService:20200212154840_9B4F59  running   18 minutes and 44.32 seconds


In [37]:
!bentoml sagemaker delete first-h2o

Successfully deleted AWS Sagemaker deployment "first-h2o"
